In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from patchify import patchify, unpatchify
from tensorflow.keras.models import load_model
import pandas as pd
import glob

In [2]:
patch_size=256
from custom_functions import predict_patches
from custom_metrics import f1,iou

In [3]:
# Define custom_objects dictionary with your metric functions
custom_objects = {'f1': f1, 'iou': iou}

# Load the model using the custom_objects parameter
root_model = load_model('root_model.h5', custom_objects=custom_objects)

## Instance segmentation from Task 5

In [4]:
def instance_segmentation(image_path, patch_size=patch_size, margin=30):
    image, preds = predict_patches(image_path, root_model, patch_size)
    predictions = unpatchify(preds, (image.shape[0], image.shape[1]))
    _, thresholded_image = cv2.threshold(predictions, 0.7, 1, cv2.THRESH_BINARY)
    thresholded_image = thresholded_image.astype('uint8')

    # Extract connected components using connectedComponentsWithStats
    retval, labels, stats, centroids = cv2.connectedComponentsWithStats(thresholded_image, 8, cv2.CV_32S)
    stats_indices = sorted(range(1, retval), key=lambda x: cv2.boundingRect((labels == x).astype(np.uint8))[0])


    instances = []
    nr_instances=0
    # Filtering elements based on criteria
    for label_index in stats_indices:
        stat = stats[label_index]
        x, y, w, h, a = stat
        if h > 400:
            if nr_instances < 5:
                # Cropping the region of interest (ROI) with a margin
                roi = thresholded_image[y-margin:y+h+margin, x-margin:x+w+margin]
                nr_instances+=1
                instances.append(roi)
                
    print(nr_instances)
    return instances


## Task 7

In [5]:
from skan import Skeleton, summarize
from skimage.morphology import skeletonize
import networkx as nx

In [6]:
# Initialize an empty list to store measurement results
results=[]

# Iterate through all image paths in sorted order
for image_path in sorted(glob.glob('Measurement_dataset/measurement_image*')):
    # Extract filename from the image path
    filename=image_path[20:-4]
    # Perform instance segmentation on the current image
    plants=instance_segmentation(image_path)
    # Iterate through each segmented region of interest (ROI) in the image
    for i, roi in enumerate(plants):
        # Convert non-zero values in the ROI to 1 (binary representation)
        roi[roi!=0]=1
        roi = roi.astype(np.uint8)
        
        # Generate the skeleton of the ROI using the skeletonize function
        skeleton = skeletonize(roi)
        # Summarize the skeleton and create a networkx graph from the summarized data
        graph=summarize(Skeleton(skeleton))
        G = nx.from_pandas_edgelist(graph, source='node-id-src', target='node-id-dst', edge_attr='branch-distance')
        
        # Extract skeleton-id from the last row of the summarized data
        skeleton_id= graph.iloc[-1]['skeleton-id']
        # Filter the graph data for the current skeleton-id
        filtered_graph = graph[(graph['skeleton-id'] == skeleton_id)]
        
        # Find the start and end points of the skeleton
        start_point = filtered_graph['node-id-src'].min()
        end_point = filtered_graph['node-id-dst'].max()
        
        # Calculate the length of the primary root using Dijkstra's algorithm
        primary_root_len = nx.dijkstra_path_length(G, start_point, end_point, weight='branch-distance')
        
        # Calculate the total length of lateral roots
        total_lateral_length = graph['branch-distance'].sum()-primary_root_len
        
        # Create a DataFrame with the measurement results for the current plant
        measurements = pd.DataFrame({'Plant ID ':[f'{filename}_plant_{i+1}'],
                                     'primary_root_len': [primary_root_len],
                                     'total_lateral_root': [total_lateral_length]})
        # Append the measurements DataFrame to the results list
        results.append(measurements)

# Concatenate all DataFrames in the results list into a single DataFrame
results_df = pd.concat(results, ignore_index=True)

5
5


In [7]:
results_df

,Plant ID,primary_root_len,total_lateral_root
0,measurement_image_1_plant_1,511.208153,0.000000
1,measurement_image_1_plant_2,1017.546248,326.859956
2,measurement_image_1_plant_3,876.563492,42.284271
3,measurement_image_1_plant_4,992.717821,279.190909
4,measurement_image_1_plant_5,875.931024,45.526912
5,measurement_image_2_plant_1,1075.960461,1020.070201
6,measurement_image_2_plant_2,1085.788889,156.308658
7,measurement_image_2_plant_3,1031.676190,700.636652
8,measurement_image_2_plant_4,932.764502,92.497475
9,measurement_image_2_plant_5,1003.333044,439.244733


In [8]:
measurements_df= pd.read_csv('Measurement_dataset\measurements_task_7.csv', delimiter=';')

In [9]:
measurements_df

,Plant ID,primary_root_len,total_lateral_root
0,measurement_image_1_plant_1,"675,321",0
1,measurement_image_1_plant_2,"1036,375","387,387"
2,measurement_image_1_plant_3,"875,321","39,284"
3,measurement_image_1_plant_4,"973,233","323,404"
4,measurement_image_1_plant_5,"874,931","43,527"
5,measurement_image_2_plant_1,"988,676","1008,09"
6,measurement_image_2_plant_2,"1103,688","137,48"
7,measurement_image_2_plant_3,"969,534","793,005"
8,measurement_image_2_plant_4,"946,25","86,569"
9,measurement_image_2_plant_5,"1004,09","431,145"


In [10]:
# Make values in the same way as in results
measurements_df['primary_root_len']  = measurements_df['primary_root_len'] .str.replace(',', '.')
measurements_df['primary_root_len'] = pd.to_numeric(measurements_df['primary_root_len'])
measurements_df['total_lateral_root']  = measurements_df['total_lateral_root'] .str.replace(',', '.')
measurements_df['total_lateral_root'] = pd.to_numeric(measurements_df['total_lateral_root'])

In [11]:
def calculate_smape(labels_df, preds_df, target_column):
    # Merge the two DataFrames on a common index or key
    merged_df = pd.merge(labels_df, preds_df, how='inner', left_index=True, right_index=True)
    # Extract actual and forecast values
    labels = merged_df[target_column + '_x']
    preds = merged_df[target_column + '_y']
    # Calculate sMAPE
    smape=100 * np.mean(np.abs(preds-labels)/((np.abs(preds) + np.abs(labels))/2))

    return smape

## Performance

In [12]:
# Calculate sMAPE for Primary root lenght
smape_primary_root = calculate_smape(measurements_df, results_df, target_column='primary_root_len')

print(f"sMAPE Primary root lenght: {round(smape_primary_root,2)}%")

sMAPE Primary root lenght: 4.95%


In [13]:
# Calculate sMAPE for Total lateral root lenght
smape_total_lateral = calculate_smape(measurements_df, results_df, target_column='total_lateral_root')

print(f"sMAPE Total lateral root lenght: {round(smape_total_lateral,2)}%")

sMAPE Total lateral root lenght: 8.7%
